# Boltzmann portfolios

### Part 2: Parametization and dynamics of weight

We develop an alternative to the traditional mean-variance
framework ("*Markowitz*" portfolios) called
***Boltzmann*** portfolios which addresses uncertainty
from the standpoint of entropy and optimal sequential decisions.
The result is a faster online algorithm which is more robust
and has no dependencies on offline convex optimization packages.

Markowitz portfolios are optimal in the arithmetic mean-variance
framework for a *single-period*. They are fragile to changing
market conditions, much like elegant battle strategies which
crumble under harsh war conditions.
In contrast, Boltzmann portfolios are designed to be
adaptive over multiple periods to maximize final wealth.
Techniques have been borrowed from Bayesian and
reinforcement learning.

[Sequential decisions to be covered in Part ?.]

*Dependencies:*

- Repository: https://github.com/rsvp/fecon235
     
*CHANGE LOG*

    2017-06-29  Investigate temperature parametization, weight dynamics.
    2017-06-28  Functions moved to ys_prtf_boltzmann module.
    2017-06-27  First version, functions numerically tested.

In [1]:
from fecon235.fecon235 import *

In [2]:
#  PREAMBLE-p6.15.1223d :: Settings and system details
from __future__ import absolute_import, print_function, division
system.specs()
pwd = system.getpwd()   # present working directory as variable.
print(" ::  $pwd:", pwd)
#  If a module is modified, automatically reload it:
%load_ext autoreload
%autoreload 2
#       Use 0 to disable this feature.

#  Notebook DISPLAY options:
#      Represent pandas DataFrames as text; not HTML representation:
import pandas as pd
pd.set_option( 'display.notebook_repr_html', False )
from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)
#  Or equivalently in config file: "InteractiveShell.display_page = True", 
#  which will display results in secondary notebook pager frame in a cell.

#  Generate PLOTS inside notebook, "inline" generates static png:
%matplotlib inline   
#          "notebook" argument allows interactive zoom and resize.

 ::  Python 2.7.13
 ::  IPython 5.1.0
 ::  jupyter_core 4.2.1
 ::  notebook 4.1.0
 ::  matplotlib 1.5.1
 ::  numpy 1.11.0
 ::  scipy 0.17.0
 ::  sympy 1.0
 ::  pandas 0.19.2
 ::  pandas_datareader 0.2.1
 ::  Repository: fecon235 v5.17.0603 devPrtf
 ::  Timestamp: 2017-06-29T12:30:44Z
 ::  $pwd: /media/yaya/virt15h/virt/dbx/Dropbox/ipy/fecon235/nb


## Introduction

***It is important to understand Part 1 of this series.
We begin by condensing its content to code,
after constructing a dataframe.***

## Download data and construct a dataframe

We retrieve the following data of daily frequency
representing equities worldwide and gold by five ETF securities: 

In [3]:
#  Convenient dictionary set in fecon235.py,
#  where key is world region, and value is its fecon235 data code:
world4d

{'America': 's4spy',
 'Emerging': 's4eem',
 'Europe': 's4ezu',
 'Gold': 's4gld',
 'Japan': 's4ewj'}

In [4]:
#  Or manually specify your own dictionary here:
prices_dic = world4d

In [5]:
#  Download data into a dataframe, alphabetically by key:
prices = groupget( prices_dic, maxi=3650 )
#  ... about ten years worth.

 ::  Retrieved from Google Finance: SPY
 ::  Retrieved from Google Finance: EEM
 ::  Retrieved from Google Finance: EZU
 ::  Retrieved from Google Finance: GLD
 ::  Retrieved from Google Finance: EWJ


## Extract the geometric mean rates

A Boltzmann portfolio seeks the best, not necessarily maximal,
weighted geometric mean rate of its underlying assets.

In [6]:
rates = gemratarr( prices )
rates

array([[  2.05],
       [-11.17],
       [-10.47],
       [  4.1 ],
       [ -4.99]])

## Global weights from covariance matrix

We now turn our attention to the weights associated with the **Global
Minimum Variance Portfolio**. Its derivation is found in Cochrane
(2005), chp. 5, p.83:

$$ \mathbf{w} = \frac{V^{-1}\mathbf{1}} { \mathbf{1}^\top V^{-1} \mathbf{1} } $$

Note that the weights are solely dependent on the covariance matrix $V$.
There are no constraints involved.

In [7]:
weighcovdata?

Signature: weighcovdata(dataframe)
Docstring: WEIGHT array (N,1) for Global Min Var Portfolio, given data.
File:      ~/Dropbox/ipy/fecon235/lib/ys_prtf_boltzmann.py
Type:      function


In [8]:
globalw = weighcovdata( prices )
globalw

array([[ 0.9032684 ],
       [-0.34952185],
       [-0.19635062],
       [ 0.46550745],
       [ 0.17709662]])

## Trim global weights

Negative weights imply the underlying assets should be shorted.
A Boltzmann portfolio only considers the weights as ***advisory***.

We may want to limit short sales at -0.3 weight, or perhaps ignore tiny
positions for rebalancing purposes in a multiple-period setting.

So we specify a threshold weight, and renormalize.

In [9]:
MIN_weight = 0.05

In [10]:
weights = rentrim( globalw, MIN_weight, 0 )
weights

array([[ 0.58430978],
       [ 0.        ],
       [ 0.        ],
       [ 0.30112927],
       [ 0.11456095]])

In [11]:
scores = weights * rates
scores

array([[ 1.19783505],
       [-0.        ],
       [-0.        ],
       [ 1.23463001],
       [-0.57165914]])

In [12]:
#  Portfolio return with trimmed weights:
np.sum(scores)

1.8608059190046435

## Introducing Boltzmann and softmax

In the field of reinforcement learning, the softmax function is used to
convert values into action probabilities.
A positive parameter T called the temperature is introduced
to divide through each value. It is a scaling operation such that
high temperatures cause corresponding actions to be equi-probable.
Low temperatures cause a greater difference in selection probability
for actions that differ in their value estimates.
At low temperatures, the probability of the action with the
highest expected reward tends to 1.

In [13]:
weighsoft??

Signature: weighsoft(weights, rates, temp, floor, level)
Source:   
def weighsoft( weights, rates, temp, floor, level ):
    '''Compute new weights transformed by softmax function.'''
    scores = weights * rates
    problist = mlearn.softmax( scores, temp )[-1]
    probs = np.array( problist ).reshape(len(problist), 1)
    #  Revise weights based on softmax probabilities:
    pweights = probs * weights
    #  Then appropriately adjust:
    return rentrim(renormalize(pweights), floor, level)
File:      ~/Dropbox/ipy/fecon235/lib/ys_prtf_boltzmann.py
Type:      function


In [14]:
#  The temperature is a hyperparameter which should be varied:
TEMPERATURE = 55

In [15]:
pweights = weighsoft(weights, rates, TEMPERATURE, MIN_weight, 0)
pweights

array([[ 0.64229991],
       [ 0.        ],
       [ 0.        ],
       [ 0.35770009],
       [ 0.        ]])

In [16]:
scores_soft = pweights * rates
scores_soft

array([[ 1.31671481],
       [-0.        ],
       [-0.        ],
       [ 1.46657037],
       [-0.        ]])

In [17]:
#  Portfolio return for Boltzmann portfolio:
np.sum(scores_soft)

2.7832851862011845

## Unify functions


```
    prices ---> cov ---> globalw
      |                    |
      |                  trimit  <-- floor
      |                  renormalize
      |                    |
      v                    v
      |                    |
    gemrat              weights
      |                    |
      |________scores______|
                 |
                 |                   Boltzmann
      temp --> softmax --> probs --> pweights

             
```

In [88]:
boltzportfolio??

Signature: boltzportfolio(dataframe, yearly=256, temp=55, floor=0.01, level=0, n=4)
Source:   
def boltzportfolio(dataframe, yearly=256, temp=55, floor=0.01, level=0, n=4):
    '''MAIN: SUMMARY of Boltzmann portfolio, rounded to n-decimal places.
       Return list where computed values are Python floats, not array type, e.g.
           [2.7833,
            [[0.6423, 2.05, 'America'],
             [0.0, -11.17, 'Emerging'],
             [0.0, -10.47, 'Europe'],
             [0.3577, 4.1, 'Gold'],
             [0.0, -4.99, 'Japan']]]
       The portfolio's geometric mean rate is included first.
       Each sub-sublist will consist of weight, rate, and key.
       The order of keys from the dataframe is preserved.
    '''
    rates = gemratarr(dataframe, yearly)
    globalw = weighcovdata(dataframe)
    weights = rentrim(globalw, floor, level)
    pweights = weighsoft(weights, rates, temp, floor, level)
    #      ---- so far should be the same as boltzweigh()
    scores = pweights * rat

In [90]:
prtf = boltzportfolio( prices, temp=TEMPERATURE, floor=MIN_weight )
prtf

[2.7813,
 [[0.6436, 2.0528, 'America'],
  [0.0, -11.1674, 'Emerging'],
  [0.0, -10.4716, 'Europe'],
  [0.3564, 4.0966, 'Gold'],
  [0.0, -4.9913, 'Japan']]]

First element is the estimated geometric mean rate of the
entire portfolio where the weights are mentioned in the sub-sublist.

## Temperature variations

Using the full *prices* dataframe, we shall vary
the temperature (proxy for our uncertainty over the estimators)
to see its effect on the Boltzmann portfolio weights.

In [85]:
#  Short sales is unrestricted, i.e. no trimming,
#  because our floor is set to -10.

for T in range(1, 421, 25):
    prtf = boltzportfolio(prices, temp=T, floor=-10, level=-10, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( T, "\t", pweights )

1 	 [0.0, -1.0, -0.0, 0.0, 0.0]
26 	 [0.07, -1.09, -0.02, 0.04, 0.0]
51 	 [0.8, -2.04, -0.21, 0.44, 0.01]
76 	 [1.14, -0.51, -0.28, 0.61, 0.04]
101 	 [2.91, -2.91, -0.69, 1.53, 0.16]
126 	 [1.87, -1.55, -0.44, 0.98, 0.13]
151 	 [1.55, -1.13, -0.36, 0.81, 0.13]
176 	 [1.39, -0.93, -0.32, 0.73, 0.13]
201 	 [1.29, -0.81, -0.29, 0.67, 0.13]
226 	 [1.23, -0.73, -0.28, 0.64, 0.14]
251 	 [1.18, -0.67, -0.27, 0.62, 0.14]
276 	 [1.15, -0.63, -0.26, 0.6, 0.14]
301 	 [1.12, -0.6, -0.25, 0.58, 0.14]
326 	 [1.1, -0.57, -0.25, 0.57, 0.15]
351 	 [1.08, -0.55, -0.24, 0.56, 0.15]
376 	 [1.07, -0.53, -0.24, 0.55, 0.15]
401 	 [1.06, -0.52, -0.24, 0.55, 0.15]


If our estimators were exact and held into the future,
at temp=1, we would entirely be shorting *Emerging* markets.
As the temperature increases, we can see wider diffusion
over the portfolio weights. Over 100 degrees seems
overly diffused.

Next, we prohibit short sales as a strategy:

In [86]:
#  Short sales is RESTRICTED,
#  because our floor is set to 0.01 and level to 0.
#  We also want to avoid tiny long positions.

for T in range(1, 421, 25):
    prtf = boltzportfolio(prices, temp=T, floor=0.01, level=0, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( T, "\t", pweights )

1 	 [0.04, 0.0, 0.0, 0.96, 0.0]
26 	 [0.62, 0.0, 0.0, 0.38, 0.0]
51 	 [0.64, 0.0, 0.0, 0.36, 0.0]
76 	 [0.65, 0.0, 0.0, 0.35, 0.0]
101 	 [0.64, 0.0, 0.0, 0.34, 0.02]
126 	 [0.64, 0.0, 0.0, 0.34, 0.02]
151 	 [0.63, 0.0, 0.0, 0.33, 0.03]
176 	 [0.63, 0.0, 0.0, 0.33, 0.04]
201 	 [0.63, 0.0, 0.0, 0.33, 0.04]
226 	 [0.62, 0.0, 0.0, 0.33, 0.05]
251 	 [0.62, 0.0, 0.0, 0.33, 0.05]
276 	 [0.62, 0.0, 0.0, 0.32, 0.06]
301 	 [0.62, 0.0, 0.0, 0.32, 0.06]
326 	 [0.61, 0.0, 0.0, 0.32, 0.06]
351 	 [0.61, 0.0, 0.0, 0.32, 0.07]
376 	 [0.61, 0.0, 0.0, 0.32, 0.07]
401 	 [0.61, 0.0, 0.0, 0.32, 0.07]


When short sales is restricted, the assets with
negative geometric mean rates will generally
be outcasts. Assets with somewhat negative rates
may get included into the portfolio as the
temperature increases.

## Evolution of weights over time

We pick a fixed temperature of 55, and see how
portfolio weights over non-overlapping periods
of 100 trading days.

In [87]:
#  Short sales is RESTRICTED,
#  because our floor is set to 0.01 and level to 0.
#  We also want to avoid tiny long positions.

for day in range(100, len(prices)-100, 100):
    prtf = boltzportfolio(prices[day-100:day], temp=55, floor=0.01, level=0, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( day, "\t", pweights )

100 	 [0.08, 0.0, 0.0, 0.85, 0.07]
200 	 [0.06, 0.0, 0.0, 0.9, 0.04]
300 	 [0.15, 0.0, 0.0, 0.18, 0.67]
400 	 [0.0, 0.0, 0.0, 0.94, 0.06]
500 	 [0.86, 0.0, 0.0, 0.14, 0.0]
600 	 [0.87, 0.0, 0.0, 0.1, 0.03]
700 	 [0.94, 0.0, 0.0, 0.01, 0.05]
800 	 [0.04, 0.0, 0.0, 0.92, 0.03]
900 	 [0.98, 0.0, 0.0, 0.02, 0.0]
1000 	 [0.22, 0.0, 0.0, 0.78, 0.0]
1100 	 [0.08, 0.0, 0.0, 0.88, 0.05]
1200 	 [0.95, 0.0, 0.0, 0.0, 0.05]
1300 	 [0.93, 0.0, 0.0, 0.05, 0.02]
1400 	 [0.84, 0.0, 0.0, 0.05, 0.11]
1500 	 [1.0, 0.0, 0.0, 0.0, 0.0]
1600 	 [0.98, 0.0, 0.0, 0.02, 0.0]
1700 	 [0.94, 0.0, 0.0, 0.05, 0.02]
1800 	 [0.94, 0.02, 0.0, 0.04, 0.0]
1900 	 [0.9, 0.0, 0.0, 0.08, 0.01]
2000 	 [0.9, 0.0, 0.0, 0.0, 0.1]
2100 	 [0.71, 0.0, 0.15, 0.0, 0.14]
2200 	 [0.07, 0.0, 0.03, 0.88, 0.03]
2300 	 [0.85, 0.0, 0.0, 0.15, 0.0]
2400 	 [0.96, 0.0, 0.0, 0.0, 0.04]


For long-only portfolios, Emerging and Europe were rarely included
in the portfolio. Largely it was a switching game between
America and Gold at major turning points.
Such shifts would seem to occur more gradually if the
periods were overlapping -- which is more realistic
in the way prices are monitored in real-time.

We shall cover optimal sequential decisions in another notebook.

---

## References

- John H. Cochrane, 2005 revised ed., *Asset Pricing*, Princeton U. Press.

- On softmax:
    - https://en.wikipedia.org/wiki/Softmax_function 
    - http://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative
    - http://cs231n.github.io/linear-classify/#softmax 
    - https://en.wikipedia.org/wiki/Reinforcement_learning 